In [1]:
from datetime import datetime
import firebirdsql
import pandas as pd

def get_sales_data():
    try:
        # Conexão com o banco de dados Firebird
        conn = firebirdsql.connect(
            host='100.64.1.10', 
            port=3050,
            database='C:\\Micromais\\mmSeculos\\BD\\SECULOS.MMDB',
            user='USER_CONSULTA',
            password='consultaseculos',
            role='apenasconsulta',
            auth_plugin_name='Legacy_Auth',
            wire_crypt=False,
            charset='ISO8859_1'  # Charset ajustado
        )
        
        print("Conexão bem-sucedida ao banco de dados!")
        
        cursor = conn.cursor()
        
        # Total diário de vendas para Jucilande e Josuilton
        total_query = '''
            SELECT 
                p.DATA,
                f.NOME,
                SUM(p.VALORCDESC) AS TOTAL_VENDAS
            FROM 
                PEDIDOVENDA p
            INNER JOIN 
                FUNCIONARIO f
            ON 
                p.CDFUNC = f.CDFUNC
            INNER JOIN 
                CLIENTE c
            ON 
                p.CDCLIENTE = c.CDCLIENTE
            WHERE 
                p.EFETIVADO = 'S'
                AND p.DATA BETWEEN '2024-12-01' AND CURRENT_DATE
                AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
            GROUP BY 
                p.DATA, f.NOME
            ORDER BY 
                p.DATA, f.NOME;
        '''
        
        cursor.execute(total_query)
        total_results = cursor.fetchall()
        
        display("Total de vendas diárias:", total_results)
        
        # Criar DataFrame para o total diário
        total_df = pd.DataFrame(total_results, columns=["Data", "Vendedor", "Total Vendas"])
        
        # Média diária de vendas por vendedor
        avg_query = '''
            SELECT 
                DIARIO.NOME,
                AVG(DIARIO.TOTAL_VENDAS) AS MEDIA_DIARIA
            FROM (
                SELECT 
                    p.DATA,
                    f.NOME AS NOME,
                    SUM(p.VALORCDESC) AS TOTAL_VENDAS
                FROM 
                    PEDIDOVENDA p
                INNER JOIN 
                    FUNCIONARIO f
                ON 
                    p.CDFUNC = f.CDFUNC
                INNER JOIN 
                    CLIENTE c
                ON 
                    p.CDCLIENTE = c.CDCLIENTE
                WHERE 
                    p.EFETIVADO = 'S'
                    AND p.DATA BETWEEN '2024-12-01' AND CURRENT_DATE
                    AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
                GROUP BY 
                    p.DATA, f.NOME
            ) DIARIO
            GROUP BY 
                DIARIO.NOME
            ORDER BY 
                DIARIO.NOME;
        '''
        
        cursor.execute(avg_query)
        avg_results = cursor.fetchall()
        
        display("Média diária de vendas por vendedor:", avg_results)
        
        # Criar DataFrame para a média diária
        avg_df = pd.DataFrame(avg_results, columns=["Vendedor", "Média Diária"])

        # Convertendo valores para float
        total_df["Total Vendas"] = total_df["Total Vendas"].astype(float)
        avg_df["Média Diária"] = avg_df["Média Diária"].astype(float)
        
        # Salvar resultados em uma planilha Excel
        with pd.ExcelWriter("arquivos/vendas_vendedores.xlsx") as writer:
            total_df.to_excel(writer, sheet_name="Total Diário", index=False)
            avg_df.to_excel(writer, sheet_name="Média Diária", index=False)
        
        print("Planilha Excel criada com sucesso: vendas_vendedores.xlsx")
        conn.close()
                
    except firebirdsql.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")


# Executa o código
get_sales_data()

Conexão bem-sucedida ao banco de dados!


'Total de vendas diárias:'

[(datetime.date(2024, 12, 2),
  'JOSUILTON MOREIRA DOS SANTOS',
  Decimal('6661.68')),
 (datetime.date(2024, 12, 2),
  'JOÃO PAULO SANTANA BATISTA',
  Decimal('1194.00')),
 (datetime.date(2024, 12, 2), 'JUCILANDE BISPO DA SILVA', Decimal('6249.03')),
 (datetime.date(2024, 12, 3),
  'JOSUILTON MOREIRA DOS SANTOS',
  Decimal('11800.08')),
 (datetime.date(2024, 12, 3), 'JUCILANDE BISPO DA SILVA', Decimal('9025.50')),
 (datetime.date(2024, 12, 4),
  'JOSUILTON MOREIRA DOS SANTOS',
  Decimal('2357.52')),
 (datetime.date(2024, 12, 4), 'JUCILANDE BISPO DA SILVA', Decimal('11474.52')),
 (datetime.date(2024, 12, 5),
  'JOSUILTON MOREIRA DOS SANTOS',
  Decimal('3853.55')),
 (datetime.date(2024, 12, 5), 'JUCILANDE BISPO DA SILVA', Decimal('8286.25')),
 (datetime.date(2024, 12, 6),
  'JOSUILTON MOREIRA DOS SANTOS',
  Decimal('3314.20')),
 (datetime.date(2024, 12, 6), 'JUCILANDE BISPO DA SILVA', Decimal('7537.88')),
 (datetime.date(2024, 12, 6), 'MANOEL VICTOR LIMA SAMPAIO', Decimal('205.00')),
 (d

'Média diária de vendas por vendedor:'

[('DANIEL ALMEIDA SPÍNOLA', Decimal('300.00')),
 ('DANIEL HENRIQUE SANTOS BEZERRA', Decimal('796.33')),
 ('JOSUILTON MOREIRA DOS SANTOS', Decimal('6319.15')),
 ('JOÃO PAULO SANTANA BATISTA', Decimal('1845.29')),
 ('JUCILANDE BISPO DA SILVA', Decimal('6865.83')),
 ('MANOEL VICTOR LIMA SAMPAIO', Decimal('279.54')),
 ('PALMIRO PEIXOTO DE OLIVEIRA', Decimal('500.00'))]

Planilha Excel criada com sucesso: vendas_vendedores.xlsx
